## Амангелді Нұрғалым, СИБ 23-01

### 4. Аутентификация и управление доступом
#### •	Задача: Создать систему аутентификации пользователей с использованием многофакторной аутентификации (например, пароль + одноразовые коды).
#### •	Цель: Защищенная аутентификация пользователей и защита от атак на пароли (brute-force)

### 1. Регистрация пользователя с хешированием пароля.

In [7]:
import bcrypt
import pyotp

# Базы данных пользователей
users_db = {}

# Функция для регистрации пользователя с хешированием пароля и генерацией секретного ключа
def register_user(username, password):
    if username in users_db:
        print("Пользователь с таким логином уже существует.")
        return
    # Хешируем пароль
    password_hash = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
    # Генерируем секретный ключ для одноразового кода
    otp_secret = pyotp.random_base32()
    # Сохраняем пользователя с хешем пароля и секретным ключом
    users_db[username] = {"password_hash": password_hash, "otp_secret": otp_secret}
    print(f"Пользователь {username} успешно зарегистрирован.")
    
# Регистрации пользователя
register_user("Nurgalym_user", "SIB2301_password")

Пользователь Nurgalym_user успешно зарегистрирован.


### 2. Проверка пароля
#### Дальше реализуем функцию для входа, которая сначала проверяет пароль.

In [8]:
# Функция для проверки пароля
def verify_password(username, password):
    user = users_db.get(username)
    if not user:
        print("Пользователь не найден.")
        return False
    
    # Проверяем пароль с хешем
    if bcrypt.checkpw(password.encode(), user["password_hash"]):
        print("Пароль подтвержден.")
        return True
    else:
        print("Неправильный пароль.")
        return False


### 3. Генерация и проверка одноразового кода
#### В следующем шаге создадим одноразовые коды с помощью "pyotp", чтобы обеспечить второй фактор аутентификации.

In [9]:
# Функция для генерации одноразового кода
def generate_otp(username):
    user = users_db.get(username)
    if not user:
        print("Пользователь не найден.")
        return None
    
    # Создаем TOTP объект
    totp = pyotp.TOTP(user["otp_secret"])
    # Генерируем код
    return totp.now()

# Функция для проверки одноразового кода
def verify_otp(username, otp_code):
    user = users_db.get(username)
    if not user:
        print("Пользователь не найден.")
        return False
    
    totp = pyotp.TOTP(user["otp_secret"])
    # Проверяем код
    if totp.verify(otp_code):
        print("Одноразовый код подтвержден.")
        return True
    else:
        print("Неправильный одноразовый код.")
        return False


### 4. Ограничение на количество попыток (защита от brute-force)
#### Вот теперь добавим счетчик неудачных попыток, чтобы заблокировать пользователя после нескольких неудачных попыток входа

In [10]:
# Добавим ограничение попыток на вход
attempts_db = {}

def authenticate_user(username, password, otp_code):
    if username not in attempts_db:
        attempts_db[username] = 0
    
    # Проверка попыток
    if attempts_db[username] >= 3:
        print("Превышено количество попыток. Попробуйте позже.")
        return False
    
    # Проверяем пароль
    if not verify_password(username, password):
        attempts_db[username] += 1
        return False
    
    # Проверяем одноразовый код
    if not verify_otp(username, otp_code):
        attempts_db[username] += 1
        return False
    
    # Сбросить счетчик при успешной аутентификации
    attempts_db[username] = 0
    print("Аутентификация пройдена.")
    return True


### 5. Завершаем процесс и тестируем программу на работоспособность

In [12]:
# Регистрация пользователя
register_user("Nurgalym_user", "SIB2301_password")

# Аутентификация: сначала вводим правильный пароль и одноразовый код
password = "SIB2301_password"
otp_code = generate_otp("Nurgalym_user")  # Получаем одноразовый код

# Проводим аутентификацию
authenticate_user("Nurgalym_user", password, otp_code)

Пользователь с таким логином уже существует.
Пароль подтвержден.
Одноразовый код подтвержден.
Аутентификация пройдена.


True

### Вывод: Глядя на результат, можно с уверенностью сказать, что система аутентификации работает правильно.
#### То есть:
#### 1. Пользователь Nurgalym_user был успешно зарегистрирован, и система создала для него хеш пароля и секретный ключ для генерации одноразовых кодов.
#### 2. При повторной попытке регистрации система выдала сообщение о том, что пользователь уже существует, что защищает от повторной регистрации.
#### 3. При входе с правильным паролем система подтвердила его как корректный.
#### 4. Введенный одноразовый код был проверен и признан верным.
#### 5. Система выдала сообщение об успешной аутентификации, что ограничивает количество попыток для защиты от brute-force атак.
